<a href="https://colab.research.google.com/github/riyanshbuktar/Final-Project-Lending-Club-Loan-data-/blob/main/Accepted_Dataset_NB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# **Import the Necessary Libraries**

In [4]:

# Import visuals
import seaborn as sns
import matplotlib.pyplot as plt

# Import pandas
import pandas as pd

# Import numpy
import numpy as np

# Import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Import scipy
from scipy.stats import boxcox
from scipy import stats

# Import Warnings
import warnings
warnings.filterwarnings('ignore')

# **Load the Dataset**

In [5]:
df_rejected= pd.read_csv('/content/drive/MyDrive/Final Project/rejected_2007_to_2018Q4.csv')

# **Understand the Dataset**

In [6]:
df_rejected.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [7]:
df_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27648741 entries, 0 to 27648740
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 1.9+ GB


In [8]:
df_rejected.shape

(27648741, 9)

In [9]:
df_rejected.size

248838669

In [10]:
df_rejected.describe()

,Amount Requested,Risk_Score,Policy Code
count,2.764874e+07,9.151111e+06,2.764782e+07
mean,1.313324e+04,6.281721e+02,6.375113e-03
std,1.500964e+04,8.993679e+01,1.127368e-01
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.800000e+03,5.910000e+02,0.000000e+00
50%,1.000000e+04,6.370000e+02,0.000000e+00
75%,2.000000e+04,6.750000e+02,0.000000e+00
max,1.400000e+06,9.900000e+02,2.000000e+00


# **Check For null values**

In [11]:
df_rejected.isnull().sum()

Amount Requested               0
Application Date               0
Loan Title                  1303
Risk_Score              18497630
Debt-To-Income Ratio           0
Zip Code                     293
State                         22
Employment Length         951355
Policy Code                  918
dtype: int64

In [12]:
df_rejected['State'] = df_rejected['State'].astype(str).str.replace('\D+', '')

In [13]:
df_rejected['Zip Code'] = df_rejected['Zip Code'].astype(str).str.replace('\D+', '')


In [14]:
df_rejected['Zip Code'].value_counts()

112    267102
770    263682
300    260429
331    245496
606    235017
        ...  
529        11
869        10
699         9
579         3
09          1
Name: Zip Code, Length: 1002, dtype: int64

In [15]:
df_rejected['State'].value_counts()

    27648741
Name: State, dtype: int64

In [16]:
# Drop the null rows since we have sufficient amount of Data
df_rejected = df_rejected.dropna(axis=0)

# **Checking for null values again**

In [17]:
# Check for null values
for col in df_rejected.columns:
      print("{} : {}".format(col, df_rejected[col].isnull().sum()))

Amount Requested : 0
Application Date : 0
Loan Title : 0
Risk_Score : 0
Debt-To-Income Ratio : 0
Zip Code : 0
State : 0
Employment Length : 0
Policy Code : 0


In [18]:
# Create a new dataframe for exploration and drop a few unnecessary columns 
df_rejected_eda = df_rejected.drop(['Policy Code','State'], axis=1)
df_rejected_eda.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,Employment Length
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481,4 years
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010,< 1 year
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212,1 year
3,6000.0,2007-05-27,waksman,698.0,38.64%,017,< 1 year
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209,< 1 year


In [19]:
# Copy this dataframe to be used later
df_rejected_eda_copy = df_rejected_eda.copy()

In [20]:
# Final check after cleaning the Data
df_rejected.shape

(8992214, 9)

In [21]:
# Final Check after cleaning the Data
df_rejected.info

<bound method DataFrame.info of           Amount Requested Application Date  ... Employment Length  Policy Code
0                   1000.0       2007-05-26  ...           4 years          0.0
1                   1000.0       2007-05-26  ...          < 1 year          0.0
2                  11000.0       2007-05-27  ...            1 year          0.0
3                   6000.0       2007-05-27  ...          < 1 year          0.0
4                   1500.0       2007-05-27  ...          < 1 year          0.0
...                    ...              ...  ...               ...          ...
27648733           30000.0       2016-12-31  ...          < 1 year          0.0
27648734            1000.0       2016-12-31  ...          < 1 year          0.0
27648736           10000.0       2016-12-31  ...          < 1 year          0.0
27648738            1200.0       2016-12-31  ...          < 1 year          0.0
27648740           15000.0       2016-12-31  ...          < 1 year          0.0

[899221

# **Lets Explore the Lending Club**

 Exploratory Data Analysis, or EDA, is an integral part of understanding the LendingClub dataset. Before moving towards classification, it's vital to become familiar with different relationships within the data. Analyzing these relationships will provide intuition about how to interpret the results of the proceeding model

# **Exploratory Data Analysis**

### **Loan Distributions**

In [22]:
# Set a new variable for the purpose of displaying the value counts of 'loan_status'
loan_status_vis = df_rejected_eda_copy['Loan Title'].value_counts()

In [23]:
pip install jupyterthemes

In [29]:
#Plot the value counts
plt.figure(figsize=(14,8))
from jupyterthemes import jtplot
jtplot.style(theme='onedork')



sns.barplot(y = loan_status_vis.index, x = loan_status_vis)
plt.title('Loan Staus Values for Lending Club',fontsize=25)
plt.ylabel('Loan Status', fontsize=15)
plt.xlabel('Count',fontsize=15)

Text(0.5, 0, 'Count')

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7efd73d4ea70> (for post_execute):


ValueError: ignored

ValueError: ignored

<Figure size 1008x576 with 1 Axes>